<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/cleaning_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregating script

## Importing libraries

In [42]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [49]:
# Storing loaded data to a pandas dataframe
import io
df1 = pd.read_excel((io.BytesIO(uploaded1['2022_08_08_file_name.xlsx'])))

In [77]:
# Checking the dataframe info
custom_list_1 = ['PERIOD', 'CUSTOMER_SEGMENT', 'CUSTOMER_ID', 'CUSTOMER_NAME', 'CHURN_CATEGORY', 'LIFETIME_DAYS_CATEOGRY', 'MARKET_SEGMENT_GROUP', 'INVOICE_CITY',
                 'FIRST_PRODUCT_TYPE', 'PORTOUT_OPERATOR_NAME', 'H_INVOICE_CUSTOMER_ID']
df1 = df1[custom_list_1]

In [ ]:
# Checking the dataframe shape
df1.tail(1)

## Aggregating Data

In [79]:
# Main data aggregation
df1_a = df1.groupby(by=custom_list_1, dropna=False).agg(CHURN_COUNT = ('H_INVOICE_CUSTOMER_ID', 'count')).reset_index() # => Key factor: Avoid dropping data if group keys contain NA values.

In [ ]:
# Interact with aggregated columns
@interact(Categories = custom_list_1)
def explore_columns(Categories):
  return pd.DataFrame(df1[Categories].value_counts(dropna=False))

In [102]:
# Creating list of aggregated fields
df1_list = [(pd.DataFrame(df1[i].value_counts(dropna=False))) for i in custom_list_1]

## Exporting results to local drive

In [117]:
# Exporting main excel file
with pd.ExcelWriter('2022_08_08_main.xlsx', engine='openpyxl') as writer:
  df1_a.to_excel(writer, sheet_name='TURNOVER', index=False)
files.download('2022_08_08_main.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [116]:
# Exporting list of excel sheets
with pd.ExcelWriter('2022_08_08_aggregations.xlsx', engine='openpyxl') as writer:
  for i in range(len(df1_list)-1):
    df1_list[i].to_excel(writer, sheet_name=f'{df1_list[i].columns[0]}', index=True)
files.download('2022_08_08_aggregations.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>